In [1]:
!pip install efficientnet #>> /dev/null/

     |████████████████████████████████| 50 kB 2.5 MB/s eta 0:00:011


In [18]:
!pip install optuna

In [19]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers
import tensorflow.keras.backend as K
import os
import efficientnet.tfkeras as efn
from keras.utils import np_utils
import optuna

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.list_physical_devices('GPU'); print(gpus)
if len(gpus)==1: strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
else: strategy = tf.distribute.MirroredStrategy()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

Mixed precision enabled


In [31]:
X = np.load("../input/pcb-image-dataset1/xtrain.npy")
y = np.load("../input/pcb-image-dataset1/ytrain.npy")

In [6]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [11]:
 conv_base = efn.EfficientNetB0(weights='imagenet',include_top=False, pooling='avg', input_shape=(300,300,3))

In [12]:
conv_base.predict(X[0].reshape(1,300,300,3)).shape

(1, 1280)

In [33]:
nos = X.shape[0]
bs = int(nos/8)


numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,1280))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 1280)
(59, 1280)
(59, 1280)
(59, 1280)
(59, 1280)
(59, 1280)
(59, 1280)
(59, 1280)


In [34]:
samples.shape, labels.shape

((472, 1280), (472,))

In [17]:
del X
del y

In [20]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

In [35]:
def optimize(trial):
    n_epochs = trial.suggest_int('n_epochs',20,100)
    n_learningrate = trial.suggest_uniform("n_learningrate",0.0001,0.1)
    n_bs = trial.suggest_int('n_bs',8,64)
    
    f = 1
    accuracies = []
    for train_index, test_index in skf.split(samples, labels):
        print('Fold {}'.format(f))
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
        y_train = np_utils.to_categorical(y_train)
        y_test = np_utils.to_categorical(y_test)
        

        K.clear_session()
        
        model = models.Sequential()
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(2, activation = 'softmax'))
        model.compile(optimizer = optimizers.Adam(lr = n_learningrate),loss = 'categorical_crossentropy',metrics = ['categorical_accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]
        with strategy.scope():
            history = model.fit(X_train, y_train, epochs = n_epochs, batch_size = n_bs, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)
            
    
        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.argmax(test_preds, axis=1)
        truth = np.argmax(y_test,axis=1)
        
        accuracies.append([confusion_matrix(truth,test_predclass)[0][0]/(truth == 0).sum(), confusion_matrix(truth,test_predclass)[1][1]/(y_test == 1).sum()])
        f+=1


    zero_acc = 0
    one_acc = 0

    for i in accuracies:
        zero_acc+= i[0]
        one_acc+=i[1]

    zero_acc/=8
    one_acc/=8

    return zero_acc+one_acc

In [36]:
from functools import partial
optimization_function = partial(optimize)

In [37]:
study = optuna.create_study(direction='maximize')

In [38]:
study.optimize(optimization_function, n_trials = 15 )

Fold 1
Epoch 1/23
13/13 [==============================] - ETA: 0s - loss: 0.7092 - categorical_accuracy: 0.60 - 0s 23ms/step - loss: 2.8236 - categorical_accuracy: 0.6441 - val_loss: 2.3401 - val_categorical_accuracy: 0.7458
Epoch 2/23
13/13 [==============================] - ETA: 0s - loss: 1.0853 - categorical_accuracy: 0.87 - 0s 8ms/step - loss: 2.7303 - categorical_accuracy: 0.6925 - val_loss: 2.4356 - val_categorical_accuracy: 0.7797
Epoch 3/23
13/13 [==============================] - ETA: 0s - loss: 2.2356 - categorical_accuracy: 0.72 - 0s 9ms/step - loss: 2.4479 - categorical_accuracy: 0.7143 - val_loss: 1.8339 - val_categorical_accuracy: 0.8136
Epoch 4/23
13/13 [==============================] - ETA: 0s - loss: 1.3613 - categorical_accuracy: 0.87 - 0s 8ms/step - loss: 1.9160 - categorical_accuracy: 0.7748 - val_loss: 1.1231 - val_categorical_accuracy: 0.7627
Epoch 5/23
13/13 [==============================] - ETA: 0s - loss: 2.2494 - categorical_accuracy: 0.60 - 0s 8ms/step - 

[I 2020-07-20 15:16:16,030] Finished trial#0 with value: 1.146410481200078 with parameters: {'n_epochs': 23, 'n_learningrate': 0.09672368131435657, 'n_bs': 33}. Best is trial#0 with value: 1.146410481200078.


Fold 1
Epoch 1/23
19/19 [==============================] - ETA: 0s - loss: 0.7685 - categorical_accuracy: 0.45 - 0s 11ms/step - loss: 1.9693 - categorical_accuracy: 0.6780 - val_loss: 1.3305 - val_categorical_accuracy: 0.7119
Epoch 2/23
19/19 [==============================] - ETA: 0s - loss: 0.8678 - categorical_accuracy: 0.77 - ETA: 0s - loss: 1.9701 - categorical_accuracy: 0.69 - 0s 7ms/step - loss: 1.9720 - categorical_accuracy: 0.6998 - val_loss: 1.8517 - val_categorical_accuracy: 0.7458
Epoch 3/23
19/19 [==============================] - ETA: 0s - loss: 2.9560 - categorical_accuracy: 0.63 - ETA: 0s - loss: 1.9746 - categorical_accuracy: 0.73 - 0s 7ms/step - loss: 1.9746 - categorical_accuracy: 0.7361 - val_loss: 2.3880 - val_categorical_accuracy: 0.6780
Epoch 4/23
19/19 [==============================] - ETA: 0s - loss: 2.6131 - categorical_accuracy: 0.72 - 0s 7ms/step - loss: 2.1576 - categorical_accuracy: 0.7337 - val_loss: 1.8524 - val_categorical_accuracy: 0.6949
Epoch 5/23
1

[I 2020-07-20 15:16:29,371] Finished trial#1 with value: 1.042562828755114 with parameters: {'n_epochs': 23, 'n_learningrate': 0.07457550288095775, 'n_bs': 22}. Best is trial#0 with value: 1.146410481200078.


Fold 1
Epoch 1/70
11/11 [==============================] - ETA: 0s - loss: 0.7850 - categorical_accuracy: 0.57 - 0s 16ms/step - loss: 0.6889 - categorical_accuracy: 0.6538 - val_loss: 0.7540 - val_categorical_accuracy: 0.7288
Epoch 2/70
11/11 [==============================] - ETA: 0s - loss: 0.6877 - categorical_accuracy: 0.76 - 0s 9ms/step - loss: 0.5649 - categorical_accuracy: 0.7579 - val_loss: 0.6131 - val_categorical_accuracy: 0.7119
Epoch 3/70
11/11 [==============================] - ETA: 0s - loss: 0.3995 - categorical_accuracy: 0.81 - 0s 9ms/step - loss: 0.5102 - categorical_accuracy: 0.7579 - val_loss: 0.5562 - val_categorical_accuracy: 0.7288
Epoch 4/70
11/11 [==============================] - ETA: 0s - loss: 0.8057 - categorical_accuracy: 0.57 - 0s 9ms/step - loss: 0.4191 - categorical_accuracy: 0.7821 - val_loss: 0.4837 - val_categorical_accuracy: 0.7966
Epoch 5/70
11/11 [==============================] - ETA: 0s - loss: 0.4685 - categorical_accuracy: 0.76 - 0s 9ms/step - 

...

[I 2020-07-20 15:19:24,181] Finished trial#12 with value: 1.1038793103448277 with parameters: {'n_epochs': 50, 'n_learningrate': 0.09801764265912491, 'n_bs': 64}. Best is trial#11 with value: 1.1652664134034678.


Fold 1
Epoch 1/21
7/7 [==============================] - ETA: 0s - loss: 0.7849 - categorical_accuracy: 0.54 - 0s 27ms/step - loss: 2.1834 - categorical_accuracy: 0.6320 - val_loss: 2.4068 - val_categorical_accuracy: 0.7119
Epoch 2/21
7/7 [==============================] - ETA: 0s - loss: 2.4604 - categorical_accuracy: 0.63 - 0s 14ms/step - loss: 1.7090 - categorical_accuracy: 0.6973 - val_loss: 1.3799 - val_categorical_accuracy: 0.7288
Epoch 3/21
7/7 [==============================] - ETA: 0s - loss: 1.2597 - categorical_accuracy: 0.78 - 0s 20ms/step - loss: 1.1607 - categorical_accuracy: 0.7554 - val_loss: 1.0179 - val_categorical_accuracy: 0.6949
Epoch 4/21
7/7 [==============================] - ETA: 0s - loss: 0.8641 - categorical_accuracy: 0.73 - 0s 23ms/step - loss: 0.8845 - categorical_accuracy: 0.7918 - val_loss: 0.7511 - val_categorical_accuracy: 0.8475
Epoch 5/21
7/7 [==============================] - ETA: 0s - loss: 0.8653 - categorical_accuracy: 0.77 - 0s 18ms/step - loss: 

[I 2020-07-20 15:19:38,119] Finished trial#13 with value: 1.0420587375803623 with parameters: {'n_epochs': 21, 'n_learningrate': 0.0754025813243251, 'n_bs': 61}. Best is trial#11 with value: 1.1652664134034678.


Fold 1
Epoch 1/53
16/16 [==============================] - ETA: 0s - loss: 0.7261 - categorical_accuracy: 0.55 - 0s 12ms/step - loss: 1.1784 - categorical_accuracy: 0.6368 - val_loss: 0.7290 - val_categorical_accuracy: 0.6949
Epoch 2/53
16/16 [==============================] - ETA: 0s - loss: 1.1538 - categorical_accuracy: 0.70 - ETA: 0s - loss: 0.8651 - categorical_accuracy: 0.74 - 0s 9ms/step - loss: 0.8651 - categorical_accuracy: 0.7458 - val_loss: 0.7283 - val_categorical_accuracy: 0.7288
Epoch 3/53
16/16 [==============================] - ETA: 0s - loss: 0.7982 - categorical_accuracy: 0.74 - ETA: 0s - loss: 0.8172 - categorical_accuracy: 0.71 - 0s 9ms/step - loss: 0.8035 - categorical_accuracy: 0.7143 - val_loss: 0.8227 - val_categorical_accuracy: 0.7288
Epoch 4/53
16/16 [==============================] - ETA: 0s - loss: 1.1221 - categorical_accuracy: 0.70 - 0s 8ms/step - loss: 1.0995 - categorical_accuracy: 0.7458 - val_loss: 1.0954 - val_categorical_accuracy: 0.6780
Epoch 5/53
1

[I 2020-07-20 15:19:52,978] Finished trial#14 with value: 1.1650496785505553 with parameters: {'n_epochs': 53, 'n_learningrate': 0.035526562148964325, 'n_bs': 27}. Best is trial#11 with value: 1.1652664134034678.


In [65]:
study.best_params

{'n_epochs': 54, 'n_learningrate': 0.0755220930502771, 'n_bs': 61}

Test Set 

In [59]:
Xtest = np.load("../input/pcb-image-dataset1/xtest.npy")
ytest = np.load("../input/pcb-image-dataset1/ytest.npy")

In [60]:
ytest = np_utils.to_categorical(ytest)

In [61]:
ytest.shape

(552, 2)

In [62]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagentest.fit(Xtest)

In [63]:
nostest = Xtest.shape[0]
bstest = int(nostest/12)


numittest = NumpyArrayIterator(
    Xtest, ytest, datagentest, batch_size=bstest, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samplestest = np.zeros(shape=(nostest,1280))
labelstest = np.zeros(shape=(nostest,2))
i = 0
for samples_batch,labels_batch in numittest:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samplestest[i*bstest:(i+1)*bstest] = f
    labelstest[i*bstest:(i+1)*bstest] = labels_batch
    i+=1
    if (i*bstest)>=nostest:
        break

(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)
(46, 1280)


In [66]:
study.best_params

{'n_epochs': 54, 'n_learningrate': 0.0755220930502771, 'n_bs': 61}

In [68]:
labelscategorical = np_utils.to_categorical(labels)
# labels.shape

In [69]:
K.clear_session()
model = models.Sequential()
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation = 'softmax'))
model.compile(optimizer = optimizers.Adam(lr = study.best_params['n_learningrate']),loss = 'categorical_crossentropy',metrics = ['categorical_accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]
with strategy.scope():
    history = model.fit(samples, labelscategorical, epochs = study.best_params['n_epochs'], batch_size = study.best_params['n_bs'], validation_data = (samplestest, labelstest), shuffle = True, callbacks = my_callbacks)

Epoch 1/54
8/8 [==============================] - ETA: 0s - loss: 0.7342 - categorical_accuracy: 0.54 - 0s 26ms/step - loss: 2.3281 - categorical_accuracy: 0.6102 - val_loss: 0.7706 - val_categorical_accuracy: 0.8768
Epoch 2/54
8/8 [==============================] - ETA: 0s - loss: 2.1502 - categorical_accuracy: 0.70 - 0s 14ms/step - loss: 1.7765 - categorical_accuracy: 0.7203 - val_loss: 0.9764 - val_categorical_accuracy: 0.7373
Epoch 3/54
8/8 [==============================] - ETA: 0s - loss: 1.4324 - categorical_accuracy: 0.72 - 0s 15ms/step - loss: 1.3375 - categorical_accuracy: 0.7352 - val_loss: 0.9626 - val_categorical_accuracy: 0.7156
Epoch 4/54
8/8 [==============================] - ETA: 0s - loss: 0.7272 - categorical_accuracy: 0.75 - 0s 15ms/step - loss: 0.8951 - categorical_accuracy: 0.7691 - val_loss: 1.3934 - val_categorical_accuracy: 0.6014
Epoch 5/54
8/8 [==============================] - ETA: 0s - loss: 1.1662 - categorical_accuracy: 0.73 - 0s 15ms/step - loss: 0.8738 

In [73]:
test_preds = model.predict(samplestest, batch_size = 20)
test_predclass = np.argmax(test_preds, axis=1)
truth = np.argmax(labelstest,axis=1)

In [74]:
accuracies_test = []
accuracies_test.append([confusion_matrix(truth,test_predclass)[0][0]/(truth == 0).sum(), confusion_matrix(truth,test_predclass)[1][1]/(truth == 1).sum()])

In [76]:
accuracies_test

[[0.711864406779661, 0.8093306288032455]]

In [ ]:
for i in conv_base.layers:
    i.trainable = False

t = [i.trainable for i in conv_base.layers]
